In [1]:
import os
import csv

import numpy as np 
import pandas as pd 
import itertools
import re

import gensim
from gensim import models
from gensim import corpora
from gensim.utils import simple_preprocess
from collections import defaultdict

from nltk.stem import SnowballStemmer   
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

In [3]:
data = pd.read_csv("data/all_english_tweets_preprocessed.csv",lineterminator='\n')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2022257 entries, 0 to 2022256
Data columns (total 25 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   Unnamed: 0      int64 
 1   Unnamed: 0.1    int64 
 2   userid          int64 
 3   username        object
 4   acctdesc        object
 5   location        object
 6   following       int64 
 7   followers       int64 
 8   totaltweets     int64 
 9   usercreatedts   object
 10  tweetid         int64 
 11  tweetcreatedts  object
 12  retweetcount    int64 
 13  text            object
 14  hashtags        object
 15  language        object
 16  coordinates     object
 17  favorite_count  int64 
 18  extractedts     object
 19  timestamp       object
 20  hashtags_words  object
 21  timestamp_H     int64 
 22  timestamp_M     int64 
 23  cleaned_text    object
 24  tokenized_text  object
dtypes: int64(11), object(14)
memory usage: 385.7+ MB


In [6]:
data.head()

,Unnamed: 0,Unnamed: 0.1,userid,username,acctdesc,location,following,followers,totaltweets,usercreatedts,...,language,coordinates,favorite_count,extractedts,timestamp,hashtags_words,timestamp_H,timestamp_M,cleaned_text,tokenized_text
0,0,357018,962369743,toonspt_,พยรลก 😉\nจะเปิดแอคเมื่อขายของเท่านั้น!!,NaN,462,7,20549,2012-11-21 13:01:19.000000,...,en,NaN,0,2022-02-24 06:51:34.430620,24-02-2022,Ukraine Russia,6,48,footage of the airport bombing in ivano franki...,"['footage', 'airport', 'bombing', 'ivano', 'fr..."
1,1,357021,1073676020,IndiaTodayFLASH,No dragging feet on news. Crisp & short news s...,NaN,117,1152610,589066,2013-01-09 12:46:38.000000,...,en,NaN,0,2022-02-24 06:51:34.403108,24-02-2022,ITVideo,6,48,ukraine mp sophia fedyna tells about the groun...,"['ukraine', 'mp', 'sophia', 'fedyna', 'tells',..."
2,2,357023,1040140500718837760,Areopagiet,Dᴀᴅ ღ Pʜɪ & Qᴜɪ /Pᴀʀᴛɴᴇʀ | Aᴛʜᴇɪsᴛ | ADD /ASS ...,EU v2.0,5008,267,9894,2018-09-13 07:29:55.000000,...,en,NaN,0,2022-02-24 06:51:34.384864,24-02-2022,Ukraine Russia,6,48,a cruise missile fired by the russian army fel...,"['cruise', 'missile', 'fired', 'russian', 'arm..."
3,3,357031,3821643918,Sicarius130,Christie // 刀剣乱舞 (izmt+mtiz for life) + 2.5D (...,Hong Kong,459,272,57829,2015-10-08 04:49:27.000000,...,en,NaN,0,2022-02-24 06:51:34.311760,24-02-2022,Ukraine Russia,6,48,spread and share you can help ukraine ukraine ...,"['spread', 'share', 'help', 'ukraine', 'ukrain..."
4,4,357034,485905284,TheAAntagonist,T.A.A | The handwriting on the wall: LIVE by t...,Omnipresent,12061,13484,813,2012-02-07 18:10:59.000000,...,en,NaN,0,2022-02-24 06:51:34.284328,24-02-2022,Putin Ukraine,6,48,now this is wrong absolutely wrong for joebide...,"['wrong', 'absolutely', 'wrong', 'joebiden', '..."


In [7]:
#I couldn't open the json file so I had to tokenize the cleaned text
porter=SnowballStemmer("english")
lmtzr = WordNetLemmatizer()

stop_words = set(stopwords.words('english'))

def tokenize(words, modulation):
    tokens = re.split(r'\W+', words)
    stems = []
    for token in tokens:
        if token not in stop_words:
            if modulation==0:
                stems.append(token)
            if modulation==1:
                stems.append(porter.stem(token))
            if modulation==2:
                stems.append(lmtzr.lemmatize(token))
    return stems

In [10]:
data['cleaned_text2'] = [tokenize(document, 0) for document in data['cleaned_text'].astype(str)]

In [11]:
data.head()

,Unnamed: 0,Unnamed: 0.1,userid,username,acctdesc,location,following,followers,totaltweets,usercreatedts,...,coordinates,favorite_count,extractedts,timestamp,hashtags_words,timestamp_H,timestamp_M,cleaned_text,tokenized_text,cleaned_text2
0,0,357018,962369743,toonspt_,พยรลก 😉\nจะเปิดแอคเมื่อขายของเท่านั้น!!,NaN,462,7,20549,2012-11-21 13:01:19.000000,...,NaN,0,2022-02-24 06:51:34.430620,24-02-2022,Ukraine Russia,6,48,footage of the airport bombing in ivano franki...,"['footage', 'airport', 'bombing', 'ivano', 'fr...","[footage, airport, bombing, ivano, frankivsk, ..."
1,1,357021,1073676020,IndiaTodayFLASH,No dragging feet on news. Crisp & short news s...,NaN,117,1152610,589066,2013-01-09 12:46:38.000000,...,NaN,0,2022-02-24 06:51:34.403108,24-02-2022,ITVideo,6,48,ukraine mp sophia fedyna tells about the groun...,"['ukraine', 'mp', 'sophia', 'fedyna', 'tells',...","[ukraine, mp, sophia, fedyna, tells, ground, s..."
2,2,357023,1040140500718837760,Areopagiet,Dᴀᴅ ღ Pʜɪ & Qᴜɪ /Pᴀʀᴛɴᴇʀ | Aᴛʜᴇɪsᴛ | ADD /ASS ...,EU v2.0,5008,267,9894,2018-09-13 07:29:55.000000,...,NaN,0,2022-02-24 06:51:34.384864,24-02-2022,Ukraine Russia,6,48,a cruise missile fired by the russian army fel...,"['cruise', 'missile', 'fired', 'russian', 'arm...","[cruise, missile, fired, russian, army, fell, ..."
3,3,357031,3821643918,Sicarius130,Christie // 刀剣乱舞 (izmt+mtiz for life) + 2.5D (...,Hong Kong,459,272,57829,2015-10-08 04:49:27.000000,...,NaN,0,2022-02-24 06:51:34.311760,24-02-2022,Ukraine Russia,6,48,spread and share you can help ukraine ukraine ...,"['spread', 'share', 'help', 'ukraine', 'ukrain...","[spread, share, help, ukraine, ukraine, russia..."
4,4,357034,485905284,TheAAntagonist,T.A.A | The handwriting on the wall: LIVE by t...,Omnipresent,12061,13484,813,2012-02-07 18:10:59.000000,...,NaN,0,2022-02-24 06:51:34.284328,24-02-2022,Putin Ukraine,6,48,now this is wrong absolutely wrong for joebide...,"['wrong', 'absolutely', 'wrong', 'joebiden', '...","[wrong, absolutely, wrong, joebiden, nato, un,..."


# LDA by day

In [12]:
#drop short tweets
print(data.shape)
data = data[data['cleaned_text2'].apply(lambda x: len(x) > 5)]
print(data.shape)

(2022257, 26)
(1885018, 26)


In [13]:
#Creating dates
import datetime
data["month-day"]=[x.strftime("%m-%d") for x in pd.to_datetime(data["timestamp"],format='%d-%m-%Y')]
data["hour-minute"]=[x.strftime(" %H:%M") for x in pd.to_datetime(data["tweetcreatedts"],format='%Y-%m-%d %H:%M:%S')]

<ipython-input-13-83bd542c902f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["month-day"]=[x.strftime("%m-%d") for x in pd.to_datetime(data["timestamp"],format='%d-%m-%Y')]
<ipython-input-13-83bd542c902f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["hour-minute"]=[x.strftime(" %H:%M") for x in pd.to_datetime(data["tweetcreatedts"],format='%Y-%m-%d %H:%M:%S')]


In [14]:
data[["timestamp","tweetcreatedts","month-day","hour-minute"]].head()

,timestamp,tweetcreatedts,month-day,hour-minute
0,24-02-2022,2022-02-24 06:48:02,02-24,06:48
1,24-02-2022,2022-02-24 06:48:02,02-24,06:48
2,24-02-2022,2022-02-24 06:48:03,02-24,06:48
3,24-02-2022,2022-02-24 06:48:03,02-24,06:48
4,24-02-2022,2022-02-24 06:48:03,02-24,06:48


Here I apply LDA for each day. Default alhpa value, and considering single tokens (proved to work better).

In [19]:
days=list(data["month-day"].unique())
topics=[]
for day in days:
    print(day)
    data_day=data[data["month-day"]==day]
    print(data_day.shape)
    texts_t = data_day.groupby('hour-minute')['cleaned_text2'].apply(list).reset_index()
    texts_t['cleaned_text2'] = [list(itertools.chain(*document)) for document in texts_t['cleaned_text2']]
    texts_t = texts_t['cleaned_text2']
    
    texts=texts_t
    #bigram = gensim.models.Phrases(texts, min_count=1, threshold=1)
    #trigram = gensim.models.Phrases(bigram[texts], threshold=1)  

    #bigram_mod = gensim.models.phrases.Phraser(bigram)
    #trigram_mod = gensim.models.phrases.Phraser(trigram)

    #texts_bigrams = [bigram_mod[doc] for doc in texts]
    #texts_trigrams = [trigram_mod[doc] for doc in texts]
    
    #texts=texts_bigrams
    frequency = defaultdict(int)
    for text in texts:
        for token in text:
            frequency[token] += 1

    texts = [
        [token for token in text if frequency[token] > 1]
        for text in texts
    ]

    dictionary = corpora.Dictionary(texts)
    print(dictionary)

    dictionary.filter_extremes(no_below = 10, no_above=0.5)
    print(dictionary)

    corpus = [dictionary.doc2bow(text) for text in texts]
    
    #alpha default is ’symmetric’: Uses a fixed symmetric prior of 1.0 / num_topics. In our case that is 0.1.
    topicnumber=3
    lda=models.LdaModel(corpus, id2word=dictionary, num_topics=topicnumber, random_state=42)
    print(lda.print_topics(topicnumber))
    topics.append((day,lda.print_topics(topicnumber)))
    print("\n")
    print("\n")

02-24
(47457, 28)
Dictionary(20881 unique tokens: ['abandon', 'abducted', 'abhorrent', 'able', 'absolutely']...)
Dictionary(5749 unique tokens: ['abandon', 'able', 'absolutely', 'accept', 'account']...)
[(0, '0.002*"chernobyl" + 0.002*"modi" + 0.001*"prices" + 0.001*"financial" + 0.001*"nfts" + 0.001*"helicopters" + 0.001*"stockmarketcrash" + 0.001*"gop" + 0.001*"means" + 0.001*"death"'), (1, '0.003*"chernobyl" + 0.001*"helicopters" + 0.001*"recent" + 0.001*"captured" + 0.001*"luhansk" + 0.001*"financial" + 0.001*"women" + 0.001*"information" + 0.001*"canada" + 0.001*"reportedly"'), (2, '0.003*"chernobyl" + 0.002*"helicopters" + 0.001*"plant" + 0.001*"financial" + 0.001*"luhansk" + 0.001*"johnson" + 0.001*"modi" + 0.001*"official" + 0.001*"prices" + 0.001*"canada"')]




02-25
(64424, 28)
Dictionary(26072 unique tokens: ['abandoned', 'abhorrent', 'absolutely', 'abuse', 'academic']...)
Dictionary(7464 unique tokens: ['abandoned', 'absolutely', 'abuse', 'academic', 'accept']...)
[(0, '0.

(91728, 28)
Dictionary(37479 unique tokens: ['abandon', 'ability', 'able', 'abolish', 'according']...)
Dictionary(11107 unique tokens: ['abandon', 'ability', 'able', 'according', 'act']...)
[(0, '0.002*"tigray" + 0.002*"save" + 0.002*"commission" + 0.002*"emmanuelmacron" + 0.002*"lives" + 0.002*"en" + 0.002*"minister" + 0.002*"women" + 0.002*"genocide" + 0.002*"vonderleyen"'), (1, '0.002*"save" + 0.002*"eucopresident" + 0.002*"emmanuelmacron" + 0.002*"minister" + 0.002*"commission" + 0.002*"foreign" + 0.002*"destroyed" + 0.002*"city" + 0.002*"next" + 0.002*"money"'), (2, '0.002*"protectuasky" + 0.002*"human" + 0.002*"bombing" + 0.002*"something" + 0.002*"vladimir" + 0.002*"missiles" + 0.002*"lives" + 0.002*"die" + 0.001*"save" + 0.001*"global"')]




03-08
(99873, 28)
Dictionary(41912 unique tokens: ['aapl', 'able', 'abundant', 'according', 'act']...)
Dictionary(12039 unique tokens: ['aapl', 'able', 'abundant', 'according', 'act']...)
[(0, '0.008*"iwd" + 0.006*"happy" + 0.003*"breakthe

[(0, '0.002*"nestle" + 0.002*"nft" + 0.002*"oil" + 0.002*"refugees" + 0.002*"daysoftigraygenocide" + 0.002*"countries" + 0.002*"lviv" + 0.002*"u" + 0.002*"emergency" + 0.002*"could"'), (1, '0.002*"attack" + 0.002*"nestle" + 0.002*"refugees" + 0.002*"lviv" + 0.002*"west" + 0.002*"nft" + 0.002*"warcrimes" + 0.002*"well" + 0.002*"international" + 0.002*"march"'), (2, '0.002*"trump" + 0.002*"nestle" + 0.002*"first" + 0.002*"crimes" + 0.002*"nft" + 0.002*"lviv" + 0.002*"warinukraine" + 0.002*"genocide" + 0.002*"weapons" + 0.002*"e"')]




03-19
(60874, 28)
Dictionary(29777 unique tokens: ['aaron', 'abaerbock', 'according', 'acting', 'actions']...)
Dictionary(8902 unique tokens: ['abaerbock', 'according', 'acting', 'actions', 'ad']...)
[(0, '0.007*"daysoftigraygenocide" + 0.006*"tigray" + 0.002*"supporthr" + 0.002*"nft" + 0.002*"million" + 0.002*"destroyed" + 0.002*"women" + 0.002*"potus" + 0.002*"days" + 0.002*"human"'), (1, '0.004*"daysoftigraygenocide" + 0.003*"tigray" + 0.002*"destroyed"

Here I apply LDA for each day using a lower value of alhpa and considering single tokens (proved to work better).

In [18]:
topics_2=[]
for day in days:
    print(day)
    data_day=data[data["month-day"]==day]
    print(data_day.shape)
    texts_t = data_day.groupby('hour-minute')['cleaned_text2'].apply(list).reset_index()
    texts_t['cleaned_text2'] = [list(itertools.chain(*document)) for document in texts_t['cleaned_text2']]
    texts_t = texts_t['cleaned_text2']
    
    texts=texts_t
    #bigram = gensim.models.Phrases(texts, min_count=1, threshold=1)
    #trigram = gensim.models.Phrases(bigram[texts], threshold=1)  

    #bigram_mod = gensim.models.phrases.Phraser(bigram)
    #trigram_mod = gensim.models.phrases.Phraser(trigram)

    #texts_bigrams = [bigram_mod[doc] for doc in texts]
    #texts_trigrams = [trigram_mod[doc] for doc in texts]
    
    #texts=texts_bigrams
    frequency = defaultdict(int)
    for text in texts:
        for token in text:
            frequency[token] += 1

    texts = [
        [token for token in text if frequency[token] > 1]
        for text in texts
    ]

    dictionary = corpora.Dictionary(texts)
    print(dictionary)

    dictionary.filter_extremes(no_below = 10, no_above=0.5)
    print(dictionary)

    corpus = [dictionary.doc2bow(text) for text in texts]
    
    #alpha default is ’symmetric’: Uses a fixed symmetric prior of 1.0 / num_topics. In our case that is 0.1.
    topicnumber=3
    lda=models.LdaModel(corpus, id2word=dictionary, num_topics=topicnumber, random_state=42,alpha=0.01)
    print(lda.print_topics(topicnumber))
    topics_2.append((day,lda.print_topics(topicnumber)))
    print("\n")
    print("\n")

02-24
(47457, 28)
Dictionary(20881 unique tokens: ['abandon', 'abducted', 'abhorrent', 'able', 'absolutely']...)
Dictionary(5749 unique tokens: ['abandon', 'able', 'absolutely', 'accept', 'account']...)
[(0, '0.002*"chernobyl" + 0.002*"modi" + 0.001*"prices" + 0.001*"financial" + 0.001*"nfts" + 0.001*"helicopters" + 0.001*"stockmarketcrash" + 0.001*"gop" + 0.001*"means" + 0.001*"death"'), (1, '0.003*"chernobyl" + 0.001*"helicopters" + 0.001*"recent" + 0.001*"captured" + 0.001*"luhansk" + 0.001*"financial" + 0.001*"women" + 0.001*"information" + 0.001*"canada" + 0.001*"reportedly"'), (2, '0.003*"chernobyl" + 0.002*"helicopters" + 0.001*"plant" + 0.001*"financial" + 0.001*"luhansk" + 0.001*"johnson" + 0.001*"modi" + 0.001*"official" + 0.001*"prices" + 0.001*"canada"')]




02-25
(64424, 28)
Dictionary(26072 unique tokens: ['abandoned', 'abhorrent', 'absolutely', 'abuse', 'academic']...)
Dictionary(7464 unique tokens: ['abandoned', 'absolutely', 'abuse', 'academic', 'accept']...)
[(0, '0.

(91728, 28)
Dictionary(37479 unique tokens: ['abandon', 'ability', 'able', 'abolish', 'according']...)
Dictionary(11107 unique tokens: ['abandon', 'ability', 'able', 'according', 'act']...)
[(0, '0.002*"tigray" + 0.002*"save" + 0.002*"commission" + 0.002*"emmanuelmacron" + 0.002*"lives" + 0.002*"en" + 0.002*"minister" + 0.002*"women" + 0.002*"genocide" + 0.002*"vonderleyen"'), (1, '0.002*"save" + 0.002*"eucopresident" + 0.002*"emmanuelmacron" + 0.002*"minister" + 0.002*"commission" + 0.002*"foreign" + 0.002*"destroyed" + 0.002*"city" + 0.002*"next" + 0.002*"money"'), (2, '0.002*"protectuasky" + 0.002*"human" + 0.002*"bombing" + 0.002*"something" + 0.002*"vladimir" + 0.002*"missiles" + 0.002*"lives" + 0.002*"die" + 0.001*"save" + 0.001*"started"')]




03-08
(99873, 28)
Dictionary(41912 unique tokens: ['aapl', 'able', 'abundant', 'according', 'act']...)
Dictionary(12039 unique tokens: ['aapl', 'able', 'abundant', 'according', 'act']...)
[(0, '0.008*"iwd" + 0.006*"happy" + 0.003*"breakth

[(0, '0.002*"nestle" + 0.002*"nft" + 0.002*"oil" + 0.002*"refugees" + 0.002*"daysoftigraygenocide" + 0.002*"countries" + 0.002*"lviv" + 0.002*"u" + 0.002*"emergency" + 0.002*"could"'), (1, '0.002*"attack" + 0.002*"nestle" + 0.002*"refugees" + 0.002*"lviv" + 0.002*"west" + 0.002*"nft" + 0.002*"warcrimes" + 0.002*"well" + 0.002*"international" + 0.002*"march"'), (2, '0.002*"trump" + 0.002*"nestle" + 0.002*"first" + 0.002*"crimes" + 0.002*"nft" + 0.002*"lviv" + 0.002*"warinukraine" + 0.002*"genocide" + 0.002*"weapons" + 0.002*"e"')]




03-19
(60874, 28)
Dictionary(29777 unique tokens: ['aaron', 'abaerbock', 'according', 'acting', 'actions']...)
Dictionary(8902 unique tokens: ['abaerbock', 'according', 'acting', 'actions', 'ad']...)
[(0, '0.007*"daysoftigraygenocide" + 0.006*"tigray" + 0.002*"supporthr" + 0.002*"nft" + 0.002*"million" + 0.002*"destroyed" + 0.002*"women" + 0.002*"potus" + 0.002*"days" + 0.002*"human"'), (1, '0.004*"daysoftigraygenocide" + 0.003*"tigray" + 0.002*"destroyed"

In [47]:
df=pd.DataFrame({"day":days,"Topics_alpha 0.1":[i[1] for i in topics],"Topics_alpha 0.01":[i[1] for i in topics_2]})

In [53]:
df["topics"]=df['Topics_alpha 0.1'].astype(str).str.replace("\d+",'').str.findall("\w+")
df["unique_words"]=[list(set(list(i))) for i in df["topics"]]
df.to_csv("topics_days.csv")
df

<ipython-input-53-c5ec8e8a8ad1>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df["topics"]=df['Topics_alpha 0.1'].astype(str).str.replace("\d+",'').str.findall("\w+")


,day,Topics_alpha 0.1,Topics_alpha 0.01,topics,unique_words
0,02-24,"[(0, 0.002*""chernobyl"" + 0.002*""modi"" + 0.001*...","[(0, 0.002*""chernobyl"" + 0.002*""modi"" + 0.001*...","[chernobyl, modi, prices, financial, nfts, hel...","[johnson, official, canada, death, means, plan..."
1,02-25,"[(0, 0.002*""sky"" + 0.001*""aircraft"" + 0.001*""c...","[(0, 0.002*""sky"" + 0.001*""aircraft"" + 0.001*""c...","[sky, aircraft, council, defending, soldier, t...","[car, voice, thank, crypto, american, sky, let..."
2,02-26,"[(0, 0.001*""ukrania"" + 0.001*""oil"" + 0.001*""tr...","[(0, 0.001*""ukrania"" + 0.001*""oil"" + 0.001*""tr...","[ukrania, oil, truth, refugees, flag, sky, tal...","[ukrania, w, thank, ukraineconflict, truth, oi..."
3,02-27,"[(0, 0.002*""sky"" + 0.002*""alert"" + 0.002*""coun...","[(0, 0.002*""sky"" + 0.002*""alert"" + 0.002*""coun...","[sky, alert, council, vonderleyen, ukranian, s...","[eucopresident, vonderleyen, racism, talk, en,..."
4,02-28,"[(0, 0.008*""sky"" + 0.005*""exclude"" + 0.001*""st...","[(0, 0.008*""sky"" + 0.005*""exclude"" + 0.001*""st...","[sky, exclude, stopwarinukraine, humanitarian,...","[stopwarinukraine, states, sky, win, stoprussi..."
5,03-01,"[(0, 0.009*""exclude"" + 0.006*""vonderleyen"" + 0...","[(0, 0.009*""exclude"" + 0.006*""vonderleyen"" + 0...","[exclude, vonderleyen, europarl, en, eucopresi...","[eucopresident, share, parliament, borisjohnso..."
6,03-02,"[(0, 0.008*""sotu"" + 0.006*""eucopresident"" + 0....","[(0, 0.008*""sotu"" + 0.006*""eucopresident"" + 0....","[sotu, eucopresident, exclude, sky, vonderleye...","[sotu, eucopresident, osce, sky, parliament, v..."
7,03-03,"[(0, 0.006*""innocent"" + 0.006*""civilian"" + 0.0...","[(0, 0.006*""innocent"" + 0.006*""civilian"" + 0.0...","[innocent, civilian, needs, defend, assistance...","[defend, sky, assistance, innocent, provide, s..."
8,03-04,"[(0, 0.002*""missiles"" + 0.002*""noflyzone"" + 0....","[(0, 0.002*""missiles"" + 0.002*""noflyzone"" + 0....","[missiles, noflyzone, missile, closethesky, co...","[eucopresident, attacked, vonderleyen, thank, ..."
9,03-05,"[(0, 0.002*""protectu"" + 0.002*""noflyzoneua"" + ...","[(0, 0.002*""protectu"" + 0.002*""noflyzoneua"" + ...","[protectu, noflyzoneua, ua, squares, parliamen...","[squares, main, anti, homeland, noflyzoneua, c..."


In [28]:
for i in range(len(topics_2)):
    print(topics[i][1]==topics_2[i][1])

True
True
False
False
False
True
False
True
False
True
False
False
True
True
True
False
False
True
False
False
True
True
True
True
False
False


Lda by day changing to 0.8 the no_above parameter in dictionary.

In [54]:
days=list(data["month-day"].unique())
topics_3=[]
for day in days:
    print(day)
    data_day=data[data["month-day"]==day]
    print(data_day.shape)
    texts_t = data_day.groupby('hour-minute')['cleaned_text2'].apply(list).reset_index()
    texts_t['cleaned_text2'] = [list(itertools.chain(*document)) for document in texts_t['cleaned_text2']]
    texts_t = texts_t['cleaned_text2']
    
    texts=texts_t
    #bigram = gensim.models.Phrases(texts, min_count=1, threshold=1)
    #trigram = gensim.models.Phrases(bigram[texts], threshold=1)  

    #bigram_mod = gensim.models.phrases.Phraser(bigram)
    #trigram_mod = gensim.models.phrases.Phraser(trigram)

    #texts_bigrams = [bigram_mod[doc] for doc in texts]
    #texts_trigrams = [trigram_mod[doc] for doc in texts]
    
    #texts=texts_bigrams
    frequency = defaultdict(int)
    for text in texts:
        for token in text:
            frequency[token] += 1

    texts = [
        [token for token in text if frequency[token] > 1]
        for text in texts
    ]

    dictionary = corpora.Dictionary(texts)
    print(dictionary)

    dictionary.filter_extremes(no_below = 10, no_above=0.8)
    print(dictionary)

    corpus = [dictionary.doc2bow(text) for text in texts]
    
    #alpha default is ’symmetric’: Uses a fixed symmetric prior of 1.0 / num_topics. In our case that is 0.1.
    topicnumber=3
    lda=models.LdaModel(corpus, id2word=dictionary, num_topics=topicnumber, random_state=42)
    print(lda.print_topics(topicnumber))
    topics.append((day,lda.print_topics(topicnumber)))
    print("\n")
    print("\n")

02-24
(47457, 28)
Dictionary(20881 unique tokens: ['abandon', 'abducted', 'abhorrent', 'able', 'absolutely']...)
Dictionary(6091 unique tokens: ['abandon', 'able', 'absolutely', 'accept', 'according']...)
[(0, '0.003*"india" + 0.002*"city" + 0.002*"belarus" + 0.002*"western" + 0.002*"u" + 0.002*"uk" + 0.002*"good" + 0.002*"government" + 0.002*"pray" + 0.002*"everyone"'), (1, '0.002*"india" + 0.002*"chernobyl" + 0.002*"government" + 0.002*"every" + 0.002*"civilians" + 0.002*"taiwan" + 0.002*"border" + 0.002*"belarus" + 0.002*"ukrainians" + 0.002*"man"'), (2, '0.003*"india" + 0.002*"minister" + 0.002*"uk" + 0.002*"u" + 0.002*"attacks" + 0.002*"good" + 0.002*"border" + 0.002*"missile" + 0.002*"chernobyl" + 0.002*"children"')]




02-25
(64424, 28)
Dictionary(26072 unique tokens: ['abandoned', 'abhorrent', 'absolutely', 'abuse', 'academic']...)
Dictionary(7738 unique tokens: ['abandoned', 'absolutely', 'abuse', 'academic', 'accept']...)
[(0, '0.003*"india" + 0.003*"zelensky" + 0.003*"air" 

(91728, 28)
Dictionary(37479 unique tokens: ['abandon', 'ability', 'able', 'abolish', 'according']...)
Dictionary(11246 unique tokens: ['abandon', 'ability', 'able', 'according', 'act']...)
[(0, '0.005*"anti" + 0.003*"stopputin" + 0.003*"eu" + 0.003*"civilians" + 0.003*"europe" + 0.003*"provide" + 0.003*"defense" + 0.003*"oil" + 0.003*"ukrainewar" + 0.003*"humanitarian"'), (1, '0.005*"anti" + 0.004*"civilians" + 0.003*"humanitarian" + 0.003*"provide" + 0.003*"stopputin" + 0.003*"news" + 0.003*"eu" + 0.003*"children" + 0.003*"oil" + 0.003*"ukrainerussiawar"'), (2, '0.004*"anti" + 0.003*"eu" + 0.003*"oil" + 0.003*"europe" + 0.003*"day" + 0.003*"provide" + 0.003*"time" + 0.003*"china" + 0.003*"russiaukrainewar" + 0.003*"news"')]




03-08
(99873, 28)
Dictionary(41912 unique tokens: ['aapl', 'able', 'abundant', 'according', 'act']...)
Dictionary(12163 unique tokens: ['aapl', 'able', 'abundant', 'according', 'act']...)
[(0, '0.008*"women" + 0.005*"internationalwomensday" + 0.004*"day" + 0.0

(64840, 28)
Dictionary(32882 unique tokens: ['abroad', 'according', 'account', 'adam', 'adamkinzinger']...)
Dictionary(9302 unique tokens: ['abroad', 'according', 'account', 'adam', 'adamkinzinger']...)
[(0, '0.005*"stop" + 0.004*"mariupol" + 0.004*"one" + 0.004*"china" + 0.004*"help" + 0.003*"biden" + 0.003*"russiaukrainewar" + 0.003*"russians" + 0.003*"children" + 0.003*"president"'), (1, '0.004*"mariupol" + 0.004*"stop" + 0.004*"help" + 0.003*"like" + 0.003*"ukrainerussianwar" + 0.003*"china" + 0.003*"russians" + 0.003*"biden" + 0.003*"one" + 0.003*"stopputin"'), (2, '0.004*"help" + 0.004*"mariupol" + 0.004*"usa" + 0.003*"like" + 0.003*"russiaukrainewar" + 0.003*"stop" + 0.003*"russians" + 0.003*"president" + 0.003*"day" + 0.003*"china"')]




03-19
(60874, 28)
Dictionary(29777 unique tokens: ['aaron', 'abaerbock', 'according', 'acting', 'actions']...)
Dictionary(8970 unique tokens: ['abaerbock', 'according', 'acting', 'actions', 'ad']...)
[(0, '0.006*"daysoftigraygenocide" + 0.005*